In [1]:
from scipy.optimize import linprog
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from gurobi import *

from dynamic_matching import DynamicMatchingBase
np.set_printoptions(precision=3)
pd.options.display.max_columns = None
from IPython.display import display, HTML
from IPython.display import display_html

### Define some utility functions

In [67]:
def in_constraint(v):
    if v[1]:
        return True
    else:
        return False

def print_tableau(matches):
    for i in range(matches.shape[2]):
        print("t=",i)
        print(matches[:,:,i])
        
def validate_allocation(allocations):
    agents_matched = allocations.sum(axis=2).sum(axis=0).sum()
    print("{} out of {} agents matched".format(agents_matched, J))
    print("{} agents have self matched".format(allocations[0,:,:].sum()))
    print("Self-matched agents are: ", np.nonzero(allocations[0,:,:].sum(axis=1))[0])

    assert agents_matched <= J, "An agent has been matched more than once"

    for i in range(I):
        for j in range(J):
            for t in range(T):
                maxit = min(T, t+d)
                assert allocations[i,j,t:maxit].sum() <= c[i], "A resource has been over allocated"

def create_tableau(I,J,T):
    valid_matches = np.zeros((I,J,T), np.int)

    #valid allocations based on arival time
    for t in range(T):
        valid_matches[:,max(0,t-d):min(t+1,J),t] = 1

    #make weights uniform across time
    pairing_weights = np.random.random(valid_matches.shape) 
    for t in range(1,pairing_weights.shape[2]):
        pairing_weights[:,:,t] = pairing_weights[:,:,0]
        
#     pairing_weights = pairing_weights.round(decimals=6)
    
    pairing_weights[0,:,:] = 1e-5
    pairing_weights[valid_matches == 0] = -1

    return valid_matches, pairing_weights

def make_alpha_mapping(I,J,T,alphas,valid_matches):
    '''Creates an index into the alpha array for each position in valid matches'''
    constraints_d, _ = dual_constraint_matrix(valid_matches,pairing_weights,I,J,T)
    constraints_d = constraints_d[:,:alphas.size]
    alpha_map = np.zeros((*valid_matches.shape,constraints_d.shape[1]),dtype=np.bool)

    cix=0
    for i in range(I):
            for j in range(J):
                for t in range(T):
                    if valid_matches[i][j][t]:
                        alpha_map[i,j,t,:] = constraints_d[cix,:]
                        cix += 1
    return alpha_map
    

To make the dual constraint matrix: 
- Create a constraint map to see which alphas/betas apply at a given location in the primal. Each valid location will correspond to a constraint in the dual, and the variables == 1 will be those in the `cmap[i][j][t]`

In [3]:
def primal_constraint_matrix(valid_matches, I,J,T):

    constraints = np.zeros((T*I+J,valid_matches.size),dtype=np.float128)
    cix = 0
    #constraints limiting to one resource allocation in the time interval
    for i in range(I):
        for t in range(T):
            constraint = np.zeros((I,J,T), np.int)
            valid_mask = constraint.copy()
            endix = min(t+k[i],T)
            valid_mask[i,:,t:endix] = 1 
            constraint[(valid_matches == 1) & (valid_mask == 1)] = 1
            constraints[cix,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))
            cix += 1

    #constraints limiting each agent to only match once            
    for j in range(J):
        constraint = np.zeros((I,J,T), np.int)
        valid_mask = constraint.copy()
        valid_mask[:,j,:] = 1

        constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
        constraints[cix+j,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))
    
    return constraints


def dual_constraint_matrix(valid_matches,pairing_weights,I,J,T):

    constraint_map = np.zeros((I,J,T,T*I+J), np.int)
    cix = 0

    #constraints limiting to one resource allocation in the time interval
    for i in range(I):
        for t in range(T):
            constraint = np.zeros((I,J,T), np.int)
            valid_mask = constraint.copy()

            endix = min(t+k[i],T)
            valid_mask[i,:,t:endix] = 1 
            constraint[(valid_matches == 1) & (valid_mask == 1)] = 1

            constraint_map[:,:,:,cix] = constraint.copy()
            cix += 1

    #constraints limiting each agent to only match once            
    for j in range(J):
        constraint = np.zeros((I,J,T), np.int)
        valid_mask = constraint.copy()
        valid_mask[:,j,:] = 1
        constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
        constraint_map[:,:,:,cix] = constraint.copy()
        cix += 1

    dual_constraint_matrix = np.zeros((valid_matches.sum(), constraint_map.shape[3]))
    inequalities = np.zeros(valid_matches.sum())
    

    cix = 0
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if valid_matches[i][j][t]:
                    dual_constraint_matrix[cix,:] = constraint_map[i,j,t,:] 
                    inequalities[cix] = pairing_weights[i,j,t]
                    cix += 1
    
    return dual_constraint_matrix, inequalities



In [4]:
def primal_solutions(pairing_weights, I, J, T):
    m = Model("dynamicmatch_primal")
    m.modelSense = GRB.MAXIMIZE
    m.setParam( 'OutputFlag', False )
    m.setParam( 'ScaleFlag', 2 )
    m.setParam( 'NumericFocus', 3)
    

    weights = pairing_weights.reshape(pairing_weights.shape[0] * pairing_weights.shape[1] * pairing_weights.shape[2])
    constraints = primal_constraint_matrix(valid_matches, I,J,T)


    keys = range(constraints.shape[1])
    variables = m.addVars(keys,
                    vtype=GRB.CONTINUOUS,
                     obj=weights,
                     name="primal",
                     lb=0)

    for cix, constraint in enumerate(constraints):
        equality = c[cix // T] if cix < T * I else 1
        m.addConstr(sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) <= equality)

    m.optimize()
    m.write('primal_formulation.lp')
    allocations = np.array([variables[var].X for var in variables], dtype=np.float128).reshape(pairing_weights.shape)

    return m.objVal, allocations


def dual_solutions(valid_matches, pairing_weights, I, J, T):
    md = Model("dynamicmatch_dual")
    md.modelSense = GRB.MINIMIZE
    md.setParam( 'OutputFlag', False )
    md.setParam( 'NumericFocus', 3)

    constraints_d, inequalities = dual_constraint_matrix(valid_matches,pairing_weights,I,J,T)
    c_d = np.ones(constraints_d.shape[1], dtype=np.float128)
    
    for ix in range(constraints_d.shape[1]):
        c_d[ix] = c[ix // T] if ix < T * I else 1
    
    keys = range(constraints_d.shape[1])
    variables = md.addVars(keys,
                    vtype=GRB.CONTINUOUS,
                    obj=c_d,
                    name="dual",
                    lb=0)

    for cix, constraint in enumerate(constraints_d):
        con = sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) >= inequalities[cix]
        md.addConstr(sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) >= inequalities[cix])

    md.optimize()
    duals = np.array([variables[var].X for var in variables],dtype=np.longdouble)
    betas = duals[duals.size - J:]
    alphas = duals[:duals.size - J]
    
    md.write('dual_formulation.lp')
    return md.objVal, alphas, betas


In [5]:
def make_alpha_tableau(valid_matches, alphas, I, T):

    alpha_tableau = np.zeros(valid_matches.shape)
    ii = 0
    it = 0
    for ax, alpha in enumerate(alphas):

        if ii < I:
            use_length = k[ax // T]
            alpha_tableau[ii,:,it:it+use_length] = alpha
            it = it+use_length

            if it >= alpha_tableau.shape[2]:
                it = 0
                ii += 1
                
    return alpha_tableau

def display_alphas_3D(valid_matches, alphas, I, T):
    amap = make_alpha_mapping(I,J,T,alphas,valid_matches)
    alpha_viz = np.zeros(valid_matches.shape)

    for i in range(I):
        for j in range(J):
            for t in range(T):
                alpha_viz[i,j,t] = np.sum(alphas[amap[i,j,t]])
    return alpha_viz

def display_alphas_2D(valid_matches, alphas, I, T):
    amap = make_alpha_mapping(I,J,T,alphas,valid_matches)
    alpha_viz = np.zeros(valid_matches.shape)

    for i in range(I):
        for j in range(J):
            for t in range(T):
                alpha_viz[i,j,t] = np.sum(alphas[amap[i,j,t]])
    return alpha_viz

def print_alpha_allocation_view(alphas, I,T):
    display(pd.DataFrame(alphas.reshape(I,T)))

### Online allocation alogrithm implementation
- At each time, for each candidate resource / utility pair, allocate if assignment term less than epsilon
- Track validity of candidate matches based on arrival/departure model, previously matched agents, and resource utilization times

In [6]:
def online_matching(I,J,T,k,c,alphas,betas,valid_matches,pairing_weights,epsilon=0):

    alpha_map = make_alpha_mapping(I,J,T,alphas,valid_matches)
    online_allocations = np.zeros(pairing_weights.shape)
    comps = np.zeros(pairing_weights.shape)
    utility = 0
    candidate_matches = valid_matches.copy()
    
    resource_uses = np.zeros((pairing_weights.shape[0],pairing_weights.shape[2]))

    for t in range(T):
        for j in range(J):
            for i in range(1,I):
                
#                 if i == 1 and j == 2 and t == 4:
#                     print('valid matches: ',valid_matches[i,j,t])
#                     print('candidate matches: ',candidate_matches[i,j,t])
#                     print('resource uses: ', resource_uses[i,t])
#                     print('ci', c[i])
                
                if candidate_matches[i,j,t] and resource_uses[i,t] < c[i]:
                    asum = np.sum(alphas[alpha_map[i,j,t] == 1])

                    comps[i,j,t] = np.abs(pairing_weights[i,j,t] - asum - betas[j])
                    #allocate if less than epsilon
                    if np.abs(pairing_weights[i,j,t] - asum - betas[j]) <= epsilon:
                        online_allocations[i,j,t] = 1
                        utility += pairing_weights[i,j,t]

                        #prevent matches with resource during time period and the same agent
                        candidate_matches[:,j,:] = 0
                        resource_uses[i,t:t+k[i]] += 1
        
            #agent hasn't been allocated, self match 
            if j == t - d and candidate_matches[0,j,t]:
                online_allocations[0,j,t] = 1
                utility += pairing_weights[0,j,t]
                candidate_matches[:,j,:] = 0
                resource_uses[0,t:t+k[i]] += 1

    return utility, online_allocations,comps


### 2D / 3D visualization functions

In [7]:
def debug_allocation_differences(online_allocs, offline_allocs, comps):
    diff = online_allocs - offline_allocs
    print("\nOnline allocated when it shouldn't (false positive)")
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if diff[i,j,t] == 1 and i > 0:
                    print("Occurs at {},{},{}. Comparison value: {}".format(i,j,t, comps[i,j,t]))
    
    
    print("\nOnline didn't allocate when it should (false negative)")
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if diff[i,j,t] == -1 and i > 0:
                    print("Occurs at {},{},{}. Comparison value: {}".format(i,j,t, comps[i,j,t]))
                    
def display_3D(tableaus_int, tableaus_float, names, c, k, T):
    '''Given an array of numpy arrays, transforms to dataframes and displays'''
    
    metadf = pd.DataFrame({
        'copies' :[c[i] for i in range(allocs.shape[0])],
        'use times' :[k[i] for i in range(allocs.shape[0])]
    })
    
    html_str = ('<th>' 
                + ''.join([f'<td style="text-align:center">{name}</td>' for name in names])
                + '</th>')

    int_ixs = len(tableaus_int)-1
    tableaus_int.extend(tableaus_float)
    
    for i in range(T):
        row_dfs = [pd.DataFrame(tableau[:,:,i]) for tableau in tableaus_int]
    
        html_str += ('<tr>' + "<td style='vertical-align:center'> t = {}".format(i) +
                     f"<td style='vertical-align:top'>{metadf.to_html(index=False, float_format='%i')}</td>" +
                     ''.join(f"<td style='vertical-align:top'> {df.to_html(index=False, float_format='%.3f') if ix > int_ixs else df.to_html(index=False,float_format='%i')}</td>"
                             for ix,df in enumerate(row_dfs)) + 
                     '</tr>')
        
    html_str = f'<table>{html_str}</table>'
    display_html(html_str, raw=True)
    
def create_allocation_vizdf(df):

    def convert_cells(cell):
        return 't='+cell.split('.')[0] if '.' in cell else ''

    df[df.columns] = df[df.columns].replace({0:np.nan}).subtract(1)
    df = df.astype(str).replace('nan', '', regex=True)
    return df.applymap(convert_cells)

def make_2D_tableau(I,J,T,pairing_weights, allocs):

    matches = np.zeros((I,J))
    match_weights = np.zeros((I,J))
    for t in range(T):
        for i in range(I):
            for j in range(J):
                if allocs[i][j][t]:
                    matches[i][j] = t+1
    match_weights = np.max(pairing_weights,axis=2)


    match_weights = np.concatenate((match_weights,betas[np.newaxis]),axis=0)
    matches = np.concatenate((matches,np.zeros((1,matches.shape[1]))),axis=0)
    
    return match_weights, matches


def display_2D(I,J,T,pairing_weights, allocs):
    
    match_weights, matches = make_2D_tableau(I,J,T,pairing_weights, allocs)

    index = ['self match'] + ['resource ' + str(i) for i in range(1, matches.shape[0]-1)] + ['betas']
    columns = ['agent '+ str(j) for j in range(matches.shape[1])]

    tabs = [matches, match_weights]
    row_dfs = [pd.DataFrame(tableau, index=index, columns=columns) for tableau in tabs]

    row_dfs[0] = create_allocation_vizdf(row_dfs[0])

    metadf = pd.DataFrame({
        'copies' :[c[i] for i in range(allocs.shape[0])] + [''],
        'use times' :[k[i] for i in range(allocs.shape[0])] + ['']
    },index=index)
    
    html_str = ('<th>' 
            + ''.join([f'<td style="text-align:center">{name}</td>' for name in ['resource info', 'allocations', 'match quality']])
            + '</th>')

    html_str += ('<tr>' + 
                 f"<td style='vertical-align:top'>{metadf.to_html(index=True, float_format='%i')}</td>" +
                 ''.join(f"<td style='vertical-align:top'> {df.to_html(index=False, float_format='%.3f') if ix > 0 else df.to_html(index=False,float_format='%i')}</td>"
                         for ix,df in enumerate(row_dfs)) + 
                 '</tr>')

    html_str = f'<table>{html_str}</table>'
    display_html(html_str, raw=True)    
    

### Online matching pseudocode: 

- For time in T
    - For agent in agents
        - For resource in resources
            - if match is valid and resource is available
                - if `weight - alphas - beta <= epsilon`:
                    - allocate here
                    - update candidate matches and resource availability
        - if agent is about to leave and is not matched
            - self match here

### Reasons for descrpencies between online and primal solutions (see below for simulations)

- The online will only self match after `d` turns of not finding a match. The optimal primal may do it earlier. This doesn't impact utility. 
- The comparison term `w[i,j,t] - alphas - beta` is zero in places where online assignment should not occur (assigns when it shouldn't).
- The comparison term `w[i,j,t] - alphas - beta` is slightly greater than zero (~`1e-17`) in places where online assignment should occur (doesnt assign when it should). Making epsilon `1e-15` fixes this issue, but then leads to a few more false positive cases where the comparison is less than `1e-15`.

This could be a numerical problem, subtle issue with the implementation, or underlying issue with online assignment




In [8]:
def check_comp_slackness(I,J,T,alphas,valid_atches):
    
    alpha_map = make_alpha_mapping(I,J,T,alphas,valid_matches)
    comp_slackness = np.zeros(valid_matches.shape)
    
    for t in range(T):
        for j in range(J):
            for i in range(1,I):
                    asum = np.sum(alphas[alpha_map[i,j,t] == 1])
                    comp_slackness[i,j,t] = (pairing_weights[i,j,t] - asum - betas[j]) * allocs[i,j,t]
                    
    return True if comp_slackness.sum() == 0 else False, comp_slackness.sum()

    

### Define situation variables

In [89]:
J = 10 # number of agents
d = 2 # wait time for each agent

I = 3 # number of resources
k=np.array([1,20,8]) # use times
c=np.array([J,2,1]) # number of copies


T = J+d # total time for matches to occur

# valid_matches, pairing_weights = create_tableau(I,J,T)

### Testing online matching

In [92]:
valid_matches, pairing_weights = create_tableau(I,J,T)
objp, allocs = primal_solutions(pairing_weights, I, J, T)
objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T)
objo, online_allocs,comps = online_matching(I,J,T,k,c,alphas,betas,valid_matches,pairing_weights,epsilon=1e-15)

In [93]:
print(objp)
print(objd)
print(objo)

3.1136427289831827
3.1136427289831827
2.9568878325105916


In [21]:
for i in range(100):

    valid_matches, pairing_weights = create_tableau(I,J,T)
    objp, allocs = primal_solutions(pairing_weights, I, J, T)
    objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T)
    objo, online_allocs,comps = online_matching(I,J,T,k,c,alphas,betas,valid_matches,pairing_weights,epsilon=1e-15)

    print(objp)
    print(objd)
    print(objo)
    print('\n')
    alpha_map = make_alpha_mapping(I,J,T,alphas,valid_matches)
    satisfied, val = check_comp_slackness(I,J,T,alphas,alpha_map)
    print(val)
    if satisfied and objp != np.round(objo,6):
        print('Complementary slackness but still not accurate')




4.7195219999999996
4.7195219999999996
4.719522000000001


-8.877752320172322e-17
5.288254
5.288254
5.288253999999999


1.6122425118038353e-17
5.012384
5.012384
5.012383999999999


-8.408326660803989e-17
5.044794
5.044794
5.044794000000001


-1.2612489991205983e-16
4.49587
4.49587
4.49587


1.2244708285508166e-17
4.8320549999999995
4.8320549999999995
4.8320549999999995


-2.310248482318214e-16
5.107422
5.107422
5.107421999999999


7.51030233012498e-17
4.547907
4.547907
4.547907


-2.3755547225515206e-16
4.697649
4.697649
3.980728000000001


2.8326984041846665e-16
4.7767965
4.7767965
4.219469000000001


1.5122672130394102e-16
4.884809
4.884809
4.707445000000002


-4.63276200171267e-17
4.187458
4.187458
3.334935


-7.0613748680802e-17
5.258889
5.258889
5.258889


-1.7204425004860996e-16
4.769225
4.769225
4.3523570000000005


9.551143513239491e-17
4.412912
4.412912
4.412912


1.2244708285508166e-17
5.404906
5.404906
4.496507000000001


-7.347163784483801e-18
5.127547
5.127547
5.12754700000

In [17]:
satisfied

False

In [69]:
pairing_weights[1,2,2] - np.sum(alphas[alpha_map[1,2,2]]) - betas[2]

-4.1633363423443370266e-17

In [70]:
0.837772 - 0.7329070000000000862 - 0.10486499999999991661

-4.163336342344337e-17

In [64]:
pairing_weights[1,2,2]

0.837772

In [65]:
np.sum(alphas[alpha_map[1,2,2]])

0.7329070000000000862

In [66]:
betas[2]

0.10486499999999991661

In [95]:
alpha_map = make_alpha_mapping(I,J,T,alphas,valid_matches)
comp_slackness = np.zeros(valid_matches.shape)

for t in range(T):
    for j in range(J):
        for i in range(1,I):
                asum = np.sum(alphas[alpha_map[i,j,t] == 1])
                comp_slackness[i,j,t] = (pairing_weights[i,j,t] - asum - betas[j]) * allocs[i,j,t]
                
                
                

In [96]:
print_tableau(comp_slackness)

t= 0
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]]
t= 1
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00
  -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
 [-0.000e+00 -4.551e-17 -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00
  -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]]
t= 2
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]]
t= 3
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]]
t= 4
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
   0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-0.000e+00 -0.000e+00 -5.551e-17 -0.000e+00 -0.000e+00 -0.000e+00
  -0.000e+00 -0.000e+00 -0.000e+00 -0.000e+00]
 [-0.000e

In [474]:
print_tableau(allocs)

t= 0
[[0. 0.]
 [0. 0.]]
t= 1
[[0. 0.]
 [0. 0.]]
t= 2
[[0. 0.]
 [1. 1.]]
t= 3
[[0. 0.]
 [0. 0.]]


In [344]:
alpha_map = make_alpha_mapping(I,J,T,alphas,valid_matches)
comp_slackness = np.zeros(valid_matches.shape)

for t in range(T):
    for j in range(J):
        for i in range(1,I):
                asum = np.sum(alphas[alpha_map[i,j,t] == 1])
                comp_slackness[i,j,t] = (pairing_weights[i,j,t] - asum - betas[j]) * allocs[i,j,t]

print(comp_slackness.sum())

-1.1771657325125584e-15


In [308]:
comp_slackness.sum()

-1.9388584162650935e-17

### Visualizing online assignment

In [370]:
validate_allocation(allocs)
validate_allocation(online_allocs)
debug_allocation_differences(online_allocs, allocs, comps)

20.0 out of 20 agents matched
15.0 agents have self matched
Self-matched agents are:  [ 1  2  3  4  5  6  7  8 10 12 13 14 16 17 18]
20.0 out of 20 agents matched
19.0 agents have self matched
Self-matched agents are:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]

Online allocated when it shouldn't (false positive)

Online didn't allocate when it should (false negative)
Occurs at 1,11,13. Comparison value: 4.5102810375396984e-17
Occurs at 1,15,17. Comparison value: 5.5102881350681254e-17
Occurs at 2,9,11. Comparison value: 1.000007097528427e-17
Occurs at 2,19,21. Comparison value: 4.551108025597356e-17


In [267]:
names=['','Primal allocations', 'Online allocations', 'Online assignment term', 'Pairing weights']
display_3D([allocs, online_allocs], [comps, pairing_weights], names, c, k, T)

Primal allocations Online allocations Online assignment term Pairing weights t = 0 
 
 
 copies 
 use times 
 
 
 
 
 4 
 1 
 
 
 2 
 2 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.808 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.686 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 1 
 
 
 copies 
 use times 
 
 
 
 
 4 
 1 
 
 
 2 
 2 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 -1.000 
 -1.000 
 
 
 0.808 
 0.405 
 -1.000 
 -1.000 
 
 
 0.686 
 0.275 
 -1.000 
 -1.000 
 
 
 t = 2 
 
 
 copies 
 use times 
 
 
 
 
 4 
 1 
 
 
 2 
 2 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 1 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 1 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.236 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 -1.000 
 
 
 0.808 
 0.405 
 0.612 
 -1.000 
 
 
 0.686 
 0.275 
 0.877 
 -1.000 
 
 
 t = 3 
 
 
 copies 
 use times 
 
 
 
 
 4 
 1 
 
 
 2 
 2 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 1 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 -1.000 
 0.000 
 0.000 
 0.000 
 
 
 -1.000 
 0.405 
 0.612 
 0.844 
 
 
 -1.000 
 0.275 
 0.877 
 0.873 
 
 
 t = 4 
 
 
 copies 
 use times 
 
 
 
 
 4 
 1 
 
 
 2 
 2 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 -1.000 
 -1.000 
 0.000 
 0.000 
 
 
 -1.000 
 -1.000 
 0.612 
 0.844 
 
 
 -1.000 
 -1.000 
 0.877 
 0.873 
 
 
 t = 5 
 
 
 copies 
 use times 
 
 
 
 
 4 
 1 
 
 
 2 
 2 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 1 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 
 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.000 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.844 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.873

In [213]:
display_2D(I,J,T,pairing_weights, allocs)


resource info allocations match quality 
 
 
 
 copies 
 use times 
 
 
 
 
 self match 
 6 
 1 
 
 
 resource 1 
 2 
 9 
 
 
 resource 2 
 1 
 5 
 
 
 betas 
 
 
 
 
 
 
 
 agent 0 
 agent 1 
 agent 2 
 agent 3 
 agent 4 
 agent 5 
 
 
 
 
 
 
 t=3 
 
 t=5 
 t=7 
 
 
 t=0 
 t=3 
 
 
 
 
 
 
 
 
 
 t=4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 agent 0 
 agent 1 
 agent 2 
 agent 3 
 agent 4 
 agent 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.858 
 0.627 
 0.144 
 0.436 
 0.341 
 0.339 
 
 
 0.026 
 0.154 
 0.008 
 0.995 
 0.850 
 0.392 
 
 
 0.239 
 0.008 
 0.000 
 0.000 
 0.000 
 0.000

In [132]:
betas

array([2.395e-01, 8.317e-03, 1.000e-05, 1.000e-05, 1.000e-05, 1.000e-05],
      dtype=float128)

Primal allocations Online allocations Online assignment term Pairing weights t = 0 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 1 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 0.627 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 0.154 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 2 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.475 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 0.627 
 0.144 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 0.154 
 0.008 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 3 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 1 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.475 
 0.183 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.627 
 0.144 
 0.436 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.154 
 0.008 
 0.995 
 -1.000 
 -1.000 
 
 
 t = 4 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 1 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 1 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.475 
 0.183 
 0.277 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.144 
 0.436 
 0.341 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.008 
 0.995 
 0.850 
 -1.000 
 
 
 t = 5 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 1 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 1 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 


In [224]:
names=['','Primal allocations', 'sum of alphas', 'Pairing weights']
alpha_viz = display_alphas(valid_matches, alphas, I, T)

display_3D([allocs], [alpha_viz, pairing_weights], names, c, k, T)

Primal allocations sum of alphas Pairing weights t = 0 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.619 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 1 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.619 
 0.619 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.145 
 0.145 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 0.627 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 0.154 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 2 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.619 
 0.619 
 0.619 
 0.000 
 0.000 
 0.000 
 
 
 0.145 
 0.145 
 0.145 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 0.627 
 0.144 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 0.154 
 0.008 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 3 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 1 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.619 
 0.619 
 0.619 
 0.000 
 0.000 
 
 
 0.000 
 0.995 
 0.995 
 0.995 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.627 
 0.144 
 0.436 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.154 
 0.008 
 0.995 
 -1.000 
 -1.000 
 
 
 t = 4 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 1 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.619 
 0.619 
 0.619 
 0.000 
 
 
 0.000 
 0.000 
 0.995 
 0.995 
 0.995 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.144 
 0.436 
 0.341 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.008 
 0.995 
 0.850 
 -1.000 
 
 
 t = 5 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 1 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.619 
 0.619 
 0.619 
 
 
 0.000 
 0.000 
 0.000 
 0.995 
 0.995 
 0.995 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.000 
 0.000 
 0.000 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.436 
 0.341 
 0.339 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.995 
 0.850 
 0.392 
 
 
 t = 6 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.619 
 0.619 
 
 
 0.000 
 0.000 
 0.000 
 0

In [225]:
print_alpha_allocation_view(alphas, I,T)

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.618757,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.145279,0.0,0.849917,0.0,0.0,0.0,0.0


In [226]:
alphas

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.619,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.145,
       0.   , 0.85 , 0.   , 0.   , 0.   , 0.   ], dtype=float128)

### Now, set up some scenarios and see if matching follows our intuitions

In [80]:
J = 6 # number of agents
d = 2 # wait time for each agent

I = 3 # number of resources
k=np.array([1,9,5]) # use times
c=np.array([J,2,2]) # number of copies


T = J+d # total time for matches to occur

valid_matches, pairing_weights = create_tableau(I,J,T)

In [84]:
objp, allocs = primal_solutions(pairing_weights, I, J, T)
objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T)

print(objp)
print(objd)

3.9524460753594965
3.9524460753594965


In [105]:
print(pd.DataFrame(np.argmax(allocs,axis=1)))
display(pd.DataFrame(np.max(pairing_weights, axis=2)))

   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  1  0  0  4  0
2  0  0  2  0  0  3  0  5


,0,1,2,3,4,5
0,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
1,0.478983,0.515323,0.118242,0.938233,0.856831,0.738432
2,0.496431,0.202180,0.295252,0.890856,0.036852,0.897752


In [ ]:
def display_3D(tableaus_int, tableaus_float, names, c, k, T):
    '''Given an array of numpy arrays, transforms to dataframes and displays'''
    
    metadf = pd.DataFrame({
        'copies' :[c[i] for i in range(allocs.shape[0])],
        'use times' :[k[i] for i in range(allocs.shape[0])]
    })
    
    html_str = ('<th>' 
                + ''.join([f'<td style="text-align:center">{name}</td>' for name in names])
                + '</th>')

    int_ixs = len(tableaus_int)-1
    tableaus_int.extend(tableaus_float)
    
    for i in range(T):
        row_dfs = [pd.DataFrame(tableau[:,:,i]) for tableau in tableaus_int]
    
        html_str += ('<tr>' + "<td style='vertical-align:center'> t = {}".format(i) +
                     f"<td style='vertical-align:top'>{metadf.to_html(index=False, float_format='%i')}</td>" +
                     ''.join(f"<td style='vertical-align:top'> {df.to_html(index=False, float_format='%.3f') if ix > int_ixs else df.to_html(index=False,float_format='%i')}</td>"
                             for ix,df in enumerate(row_dfs)) + 
                     '</tr>')
        
    html_str = f'<table>{html_str}</table>'
    display_html(html_str, raw=True)
    